<a href="https://colab.research.google.com/github/vohai2003/Pythoncode/blob/main/BTKN4_1_Kfold_continuous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/mnt")

Mounted at /mnt


In [2]:
import numpy
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [3]:
D = numpy.genfromtxt("/mnt/My Drive/Dataset/ex1data2.txt",delimiter=",")
scaler = MinMaxScaler()
scaler.fit(D)
X,y = D[:,:-1], D[:,-1]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=30)

In [4]:
kf = KFold(n_splits=10)
model = LinearRegression()
step = 0

In [5]:
for train_index, val_index in kf.split(X=X_train,y=y_train):
  step += 1
  X_singtrain, X_singval = X_train[train_index], X_train[val_index]
  y_singtrain, y_singval = y_train[train_index], y_train[val_index]
  model.fit(X_singtrain,y_singtrain)
  yhat = model.predict(X_singval)
  yhat_test = model.predict(X_test)
  val_loss = mean_squared_error(y_singval,yhat)
  test_loss = mean_squared_error(y_test,yhat_test)
  print("Step: {:<10} val_loss: {:<10} test_loss: {:<10}".format(step,val_loss,test_loss))

Step: 1          val_loss: 5202734118.655806 test_loss: 4537002581.073696
Step: 2          val_loss: 4229536282.3030725 test_loss: 4816230593.106419
Step: 3          val_loss: 2088826363.3395078 test_loss: 4759561320.249105
Step: 4          val_loss: 9465348099.063633 test_loss: 5081299083.038312
Step: 5          val_loss: 21216562279.370193 test_loss: 8080089741.594469
Step: 6          val_loss: 4194729894.4893346 test_loss: 4655711144.2112255
Step: 7          val_loss: 2147458614.0355954 test_loss: 4550173760.788771
Step: 8          val_loss: 17527112065.8073 test_loss: 4545374322.726393
Step: 9          val_loss: 23122665.267594624 test_loss: 4680221691.5304365
Step: 10         val_loss: 2208874234.9507813 test_loss: 4699095342.43166


Hàm thực hiện K-fold CV

In [7]:
def Kfold_continuous(model,k:int,X,y):
  kf = KFold(n_splits=k)
  step = 0
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=30,random_state=140)
  for train_index, val_index in kf.split(X=X_train,y=y_train):
    step += 1
    X_singtrain, X_singval = X_train[train_index], X_train[val_index]
    y_singtrain, y_singval = y_train[train_index], y_train[val_index]
    model.fit(X_singtrain,y_singtrain)
    yhat = model.predict(X_singval)
    val_loss = mean_squared_error(y_singval,yhat)
  yhat_test = model.predict(X_test)
  test_loss = mean_squared_error(y_test,yhat_test)
  return test_loss

Thực hiện K-fold CV với mô hình Lasso để điều chỉnh hệ số $\alpha$ của mô hình

In [11]:
patient = 30
working = True
alpha = 0
memory = numpy.array([1e+99]*patient)
min = 1e+99
min_alpha = -1
while working:
  lasso_model = Lasso(alpha = alpha)
  eval = Kfold_continuous(lasso_model,10,X,y)
  if eval < min:
    min = eval
    min_alpha = alpha
  memory = numpy.append(memory[1:],eval)
  # đoạn lệnh dưới đây dùng để dừng vòng lặp khi gặp giá trị alpha phù hợp
  if numpy.sum(memory == min) == 0: #tìm giá trị của min trong dãy dữ liệu nhớ
    working = False
  print("alpha:",alpha,"eval_score:",eval)
  alpha += 1
print("Suitable alpha:",min_alpha)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.814e+10, tolerance: 2.190e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_

alpha: 0 eval_score: 5011736142.941385
alpha: 1 eval_score: 3616419100.95264
alpha: 2 eval_score: 5307402612.830865
alpha: 3 eval_score: 4518924107.648428
alpha: 4 eval_score: 4031193184.728183
alpha: 5 eval_score: 5007363540.648159
alpha: 6 eval_score: 4986609460.909272
alpha: 7 eval_score: 4729881387.005136
alpha: 8 eval_score: 5034457764.1383705
alpha: 9 eval_score: 9975931910.985558
alpha: 10 eval_score: 5181134900.486481
alpha: 11 eval_score: 5900224503.523784
alpha: 12 eval_score: 4799548743.7085705
alpha: 13 eval_score: 7628060798.993717
alpha: 14 eval_score: 4618861721.5142765
alpha: 15 eval_score: 4785902022.372976
alpha: 16 eval_score: 5143799557.596684
alpha: 17 eval_score: 6254212643.96902
alpha: 18 eval_score: 7044902489.26907
alpha: 19 eval_score: 5036198989.977551
alpha: 20 eval_score: 5368137440.590323
alpha: 21 eval_score: 5936333455.014178
alpha: 22 eval_score: 6018565992.649867
alpha: 23 eval_score: 4723895046.752321
alpha: 24 eval_score: 6506887091.350585
alpha: 25 